In [ ]:
# src/1_ingestao.py
import os
import requests
import boto3
from botocore.exceptions import ClientError
from tqdm import tqdm
import logging
from dotenv import load_dotenv

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def configurar_ambiente():
    if not load_dotenv('.env'):
        logger.warning("Arquivo .env não encontrado.")

    # Verifica e solicita configurações faltantes
    verificar_configuracao('AWS_ACCESS_KEY_ID', "AWS Access Key ID")
    verificar_configuracao('AWS_SECRET_ACCESS_KEY', "AWS Secret Access Key")
    verificar_configuracao('S3_BUCKET', "Nome do Bucket S3")

def verificar_configuracao(variavel: str, descricao: str):
    if not os.getenv(variavel):
        print(f"\nConfiguração não encontrada: {descricao}")
        valor = input(f"Por favor, informe o valor para {descricao}: ").strip()
        
        os.environ[variavel] =  valor
        logger.info(f"Configuração {variavel} salva na seção atual")

def download_de_nyc(url: str) -> bytes:
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        logger.error(f"Erro no download: {e}")
        raise

def upload_para_s3(conteudo: bytes, chave: str):
    try:
        s3 = boto3.client(
            's3',
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
        )
        s3.put_object(
            Bucket=os.getenv('S3_BUCKET'),
            Key=chave,
            Body=conteudo,
            ACL='bucket-owner-full-control'
        )
        logger.info(f"Arquivo {chave} carregado com sucesso no bucket {os.getenv('S3_BUCKET')}")
    except ClientError as e:
        logger.error(f"Erro S3: {e.response['Error']['Message']}")
        raise

def principal():
    # Configura ambiente
    configurar_ambiente()
    
    # Configurações
    URL_BASE = "https://d37ci6vzurychx.cloudfront.net/trip-data"
    PREFIXO_S3 = "raw"
    
    # Meses de interesse (Jan-Mai 2023)
    meses = [f"{i:02d}" for i in range(1, 6)]
    tipos_taxi = ["yellow", "green"]
    
    for mes in tqdm(meses, desc="Processando meses"):
        for tipo in tipos_taxi:
            try:
                nome_arquivo = f"{tipo}_tripdata_2023-{mes}.parquet"
                url = f"{URL_BASE}/{nome_arquivo}"
                
                # Download
                conteudo_arquivo = download_de_nyc(url)
                
                # Upload
                chave_s3 = f"{PREFIXO_S3}/{tipo}/{nome_arquivo}"
                upload_para_s3(conteudo_arquivo, chave_s3)
                
            except Exception as e:
                logger.error(f"Falha no processamento do mês {mes} para o taxi {tipo}: {str(e)}")
                continue

if __name__ == "__main__":
    principal()